[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/audiogen-colab/blob/main/audiogen_colab.ipynb)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install -q git+https://github.com/camenduru/audiocraft

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/audo/audiogen-medium/resolve/main/compression_state_dict.bin -d /content/model -o compression_state_dict.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/audo/audiogen-medium/resolve/main/state_dict.bin -d /content/model -o state_dict.bin

import gradio as gr
from audiocraft.models import AudioGen
from audiocraft.data.audio import audio_write

model = AudioGen.get_pretrained('/content/model')

def generate(description, top_k=250, top_p=0.0, temperature=1.0, duration=10, cfg_coef=3.0):
    descriptions = [description]
    model.set_generation_params(use_sampling=True, top_k=top_k, top_p=top_p, temperature=temperature, duration=duration, cfg_coef=cfg_coef)
    output = model.generate(descriptions, progress=False)
    audio_write(f'out', output[0].cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)
    return "/content/out.wav"

block = gr.Blocks()
with block:
    with gr.Group():
      description = gr.Textbox(label="text",value="A cat meowing")
      with gr.Row():
        top_k = gr.Slider(label="top_k", value=250, step=1, maximum=500)
        top_p = gr.Slider(label="top_p",value=0.0, step=0.1, maximum=10)
        temperature = gr.Slider(label="temperature",value=1.0, step=0.1, maximum=10)
        duration = gr.Slider(label="duration",value=10, step=0.1, maximum=10)
        cfg_coef = gr.Slider(label="cfg_coef",value=3.0, step=0.1, maximum=10)
      btn = gr.Button("Generate")
      audio_out = gr.Audio()
    btn.click(generate, inputs=[description, top_k, top_p, temperature, duration, cfg_coef], outputs=audio_out)
block.launch(debug=True)